In [ ]:
import json
import mlflow

from collections import defaultdict
from constants import TRACKING_SERVER_URI

mlflow.set_tracking_uri(TRACKING_SERVER_URI)

In [ ]:
experiment_name = "TF-Grupo1-CompML"
experiment = mlflow.get_experiment_by_name(experiment_name)

runs_list = mlflow.search_runs(
    experiment_ids=[experiment.experiment_id],
    output_format="list"
)

hyperparams_runs = {'hidden_size': [], 'dropout': [], 'lr': [], 'batch_size': []}
for run in runs_list:
    run_data = run.data
    run_info = run.info
    status = run_info.status
    if status != "FINISHED":
        continue

    run_params = run_data.params    # type: dict

    train_params = run_params['train_params'].replace("'", '"')
    train_params = json.loads(train_params)

    for k, v in train_params.items():
        if k in hyperparams_runs:
            hyperparams_runs[k].append(v)

import pandas as pd
hyperparams_df = pd.DataFrame(hyperparams_runs)
hyperparams_df.value_counts()

In [ ]:
for hyperparam in hyperparams_df.columns:
    print(hyperparams_df[hyperparam].value_counts() / len(hyperparams_df) * 100)
    print()

In [ ]:
experiment_name = "TF-Grupo3-CompML"
experiment = mlflow.get_experiment_by_name(experiment_name)

runs_list = mlflow.search_runs(
    experiment_ids=[experiment.experiment_id],
    output_format="list"
)

date_runs = defaultdict(list)
for run in runs_list:
    run_data = run.data
    run_info = run.info

    run_params = run_data.params    # type: dict
    run_model_arch = run_params['model_arch']
    if run_model_arch != "conv":
        continue

    run_status = run_info.status
    if run_status != "FINISHED":
        continue

    run_name = run_info.run_name
    run_date = run_name.split("_")[1].split("-")[0]
    date_runs[run_date].append(run_params['f1_score_test_after_training'])

date_runs_today = date_runs['20250529']
date_runs_before = date_runs['20250527'] + date_runs['20250528']
print(f"Runs today: {len(date_runs_today)}")
print(f"Runs before today: {len(date_runs_before)}")

avg_today = sum(float(x) for x in date_runs_today) / len(date_runs_today)
avg_before = sum(float(x) for x in date_runs_before) / len(date_runs_before)
print(f"Average F1 score today: {avg_today:.4f}")
print(f"Average F1 score before today: {avg_before:.4f}")